# Research Track 2 - Assignment 2

The package **rt2_assignment1** contains a modular structure able to simulate a vehicle in  a Gazebo simulation that can either reach a series of random goals or move following the user teleoperation.
The original ``user_interface`` node has been replaced with the new interface implemented below. The new user interface allows to get a random goal for the robot to reach, teleoperate it, and also shows some feedbacks through a graphical interface.

## Dependencies

## Running the code
This is a ROS package, so make sure to source it the bashrc.  
All the necessary nodes can be run through the command:  
    ```
        rosalunch rt2_assignment1 sim.launch       
    ```  
The graphical user interface is available on jupyter notebook an can be run through:  
    ```  
        jupyter notebook --allow-root  
    ```  
in the folder **rt2_assignment1/src/jupyter**. 

## Package architecture
The package is composed by two cpp nodes
- position_server
- state_machine

and two python nodes

- user_interface
- go_to_point

## User interface
The Jupyter user interface allows to send commands to the vehicle in the Gazebo environment as well as getting visual feedbacks.

In [ ]:
%matplotlib widget
import numpy as np
import jupyros as jr
import rospy
from geometry_msgs.msg import Twist
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import actionlib
import actionlib.msg
from actionlib_msgs.msg import GoalID
import rt2_assignment1.msg
import time
from rt2_assignment1.srv import Command
import matplotlib.pyplot as plt
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist
import ipywidgets as widgets
import time

teleop = True   # flag for canceling goals in the teleop mode
reached = 0     # number of reached goals
canceled = 0    # number of canceled goals
start_t = 0     # goal reching start time instance 
stop_t = 0      # goal reched time instance
elapsed_t = []  # list of the amount of time taken to reach each goal 

'''
    Callback of the /reaching_goal/result topic
    if the goal has been reached succesfully, the global variable "reached" is incremented,
    the variable "stop_t" is updated with the current time and the duration is computed and appended in the
    "elapsed_t" list. '''
def result_callback(result):
    global reached, start_t, stop_t, elapsed_t
    if(result.result.result):
        reached +=1
        stop_t = time.time()
        elapsed = stop_t - start_t
        elapsed_t.append(elapsed)
        
        
rospy.init_node('user_interface')

pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
sub = rospy.Subscriber('/reaching_goal/result',rt2_assignment1.msg.PlanningActionResult,result_callback)
ui_client = rospy.ServiceProxy('/user_interface', Command)
msg = Twist()

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


### Random goal interface
The "start and stop" command interface from the original package has been reimplemented in the notebook and uses buttons from the **ipywidgets** library. As the former one, it is the client to the ``/user_interface`` service that communicates with the ``state_machine`` node.  
The Start button callback publishes a "start" Command message, updates the global variable "start_t" with the current time instance.  
The Stop button callback publishes a "stop" Command message, updates the global variable "canceled".


In [ ]:
#Random position behaviour
start_button = widgets.Button(description="start")
output = widgets.Output()
display(start_button, output)

stop_button = widgets.Button(description="stop")
output = widgets.Output()
display(stop_button, output)

def on_start_button_clicked(b):
    global teleop, start_t
    with output:
        ui_client("start")
        start_t = time.time()
        teleop = False
        
def on_stop_button_clicked(b):
    global teleop, canceled
    with output:
        ui_client("stop")
        canceled +=1
        teleop = False

start_button.on_click(on_start_button_clicked)
stop_button.on_click(on_stop_button_clicked)

### Teleoperation interface
This second part of the interface allows to teleoperate the robot through four directional buttons and one stop button. The directional buttons call a class of functions that publish a ``Twist`` message to the ``/cmd_vel`` topic with the appropriate velocity component. In order to switch fromthe previous "reach a random goal" modality to this one it is sufficient to push one of the five button, as the active goal, if any, will be canceled upon calling of any function of the class ``Instruction()``. From this isnterface it is also possible to change the linear and angular velocity of the robot through two sliders that range from 0.0 to 2.0. The values are published on the topic ``/cmd_vel`` upon sliding the widget.

In [2]:
# Teleoperation
## Sliders
lin_vel = widgets.FloatSlider(min=0, max=2, value=1.0, description='Linear vel:')
lin_out = widgets.Output()
display(lin_vel, lin_out)

ang_vel = widgets.FloatSlider(min=0, max=2, value=1.0, description='Angular vel:')
ang_out = widgets.Output()
display(ang_vel, ang_out)

def on_lin_change(change):
    with lin_out:
        global msg
        msg.linear.x = change['new']
        pub.publish(msg)
        
def on_ang_change(change):
    with ang_out:
        global msg
        msg.angular.z = change['new']
        pub.publish(msg)

        
lin_vel.observe(on_lin_change,
names='value')

ang_vel.observe(on_ang_change,
names='value')


## Buttons
b_up = Button(description='^',
    layout=Layout(width='auto', align="center", grid_area='b_u'),
    style=ButtonStyle(button_color='lightblue'))
b_down = Button(description='v',
    layout=Layout(width='auto', grid_area='b_d'),
    style=ButtonStyle(button_color='lightblue'))
b_left = Button(description='<',
    layout=Layout(width='auto', grid_area='b_l'),
    style=ButtonStyle(button_color='lightblue'))
b_right = Button(description='>',
    layout=Layout(width='auto', grid_area='b_r'),
    style=ButtonStyle(button_color='lightblue'))
b_stop = Button(description='stop',
    layout=Layout(width='auto', grid_area='b_s'),
    style=ButtonStyle(button_color='red'))

GridBox(children=[b_up, b_down, b_left, b_right, b_stop],
    layout=Layout(
        width='40%',
        grid_template_rows='auto auto auto auto auto',
        grid_template_columns='33% 33% 33%',
        grid_template_areas='''
        " . b_u . "
        "b_l b_d b_r "
        " . b_s . "
        "lin . ."
        "ang . ."
        ''')
)

FloatSlider(value=1.0, description='Linear vel:', max=2.0)

Output()

FloatSlider(value=1.0, description='Angular vel:', max=2.0)

Output()

GridBox(children=(Button(description='^', layout=Layout(grid_area='b_u', width='auto'), style=ButtonStyle(butt…

In [3]:
def cancel_goal():
    global teleop, canceled
    if not teleop:
        ui_client("stop")
        teleop = True
        canceled +=1
        
class Instruction:
    def up(self, event):
        global msg, ac
        cancel_goal()
        msg.linear.x = 1*lin_vel.value
        pub.publish(msg)
        
    def down(self, event):
        global msg, ac
        cancel_goal()
        msg.linear.x = -1*lin_vel.value
        pub.publish(msg)
        
    def left(self, event):
        global msg, ac
        cancel_goal()
        msg.angular.z = -1*ang_vel.value
        pub.publish(msg)
        
    def right(self, event):
        global msg, ac
        cancel_goal()
        msg.angular.z = 1*ang_vel.value
        pub.publish(msg)
    
    def stop(self, event):
        global msg, ac
        cancel_goal()
        msg.linear.x = 0
        msg.angular.z = 0
        pub.publish(msg)

callback = Instruction()
b_up.on_click(callback.up)
b_down.on_click(callback.down)
b_left.on_click(callback.left)
b_right.on_click(callback.right)
b_stop.on_click(callback.stop)

## Graphical feedbacks

### Imposed vs actual velocities
The graph plots in real time the actual linear and angular velocities, provided by the ``/odom`` topic, against the imposed linear and angular velocities, provided by the ``/cmd_vel`` topic.

In [14]:
# Odometry vs cmd_vel Velocities graph

timeline = list(range(0, 10))
start_timer = time.time()

fig, ax = plt.subplots()
# plots
lin_odom, = ax.plot([], [], color='b', label='odom lin vel') 
ang_odom, = ax.plot([], [], color='r', label='odom ang vel') 
lin_cmd, = ax.plot([], [], color='b', linestyle='--', label='cmd lin vel') 
ang_cmd, = ax.plot([], [], color='r', linestyle='--', label='cmd ang vel')
# lables
ax.set_xlabel('time (s)') 
ax.set_ylabel('velocities')
ax.set_title("Odometry vs cmd_vel")
plt.legend(loc='lower left')
plt.show()

lo_data = []
ao_data = []
lc_data = []
ac_data = []

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def init1():
    lin_odom.set_data([], [])
    ang_odom.set_data([], [])
    lin_cmd.set_data([], [])
    ang_cmd.set_data([], [])
    return (lin_odom, ang_odom, lin_cmd, ang_cmd)


def odom_twist_callback(msg):
    if len(timeline)>10:
        timeline.pop(0)
    if len(lo_data)>10:
        lo_data.pop(0)
    if len(ao_data)>10:
        ao_data.pop(0)
    lo_data.append(msg.twist.twist.linear.x)
    ao_data.append(msg.twist.twist.angular.z)
    timeline.append(time.time()-start_timer)
    
jr.subscribe('/odom', Odometry, odom_twist_callback)

Removing previous callback, only one redirection possible right now


In [16]:
def cmd_callback(msg):
    if len(timeline)>10:
        timeline.pop(0)
    if len(lc_data)>10:
        lc_data.pop(0)
    if len(ac_data)>10:
        ac_data.pop(0)
    lc_data.append(msg.linear.x)
    ac_data.append(msg.angular.z)
    timeline.append(time.time()-start_timer)
    
jr.subscribe('/cmd_vel', Twist, cmd_callback)

Removing previous callback, only one redirection possible right now


In [17]:
def animate1(i):
    lin_odom.set_data(timeline, lo_data)
    ang_odom.set_data(timeline, ao_data)
    lin_cmd.set_data(timeline, lc_data)
    ang_cmd.set_data(timeline, ac_data)
    ax.axis([timeline[0]-0.2,timeline[9]+0.2,-0.5,0.5])   
    return (lin_odom, ang_odom, lin_cmd, ang_cmd)

anim1 = animation.FuncAnimation(fig, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

### Real time position feedback
The graph below and it is live updated and it shows the position of the robot in the plane xy thanks to the data provided by the ``/odom`` topic.

In [8]:
# Real time position graph
fig1, ax1 = plt.subplots()

ax1.set_xlim(( -5, 5))
ax1.set_ylim((-5, 5))
ax1.set_title("Odometry vs cmd_vel") 

#arrow = ax.text(xpos, ypos, "↑", color='r', size=15, rotation=ang)

line, = ax1.plot([], [], 'ro')
x_data=[]
y_data=[]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def init2():
    line.set_data([], [])
    return (line,)

def odom_pose_callback(msg):
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)
    
def animate2(i):
    line.set_data(x_data, y_data)
    return (line,)

jr.subscribe('/odom', Odometry, odom_pose_callback)

Removing previous callback, only one redirection possible right now


In [10]:
anim = animation.FuncAnimation(fig, animate2, init_func=init2,
                               frames=100, interval=20, blit=True)

### Reached and canceled targets
This bar plot shows the number of reached and canceled goals. The global variable ``reached`` is updated in the ``result_callback()`` function, which is called every time a message from  the topic ``/reaching_goal/result`` is received.  The global variable ``canceled`` is updated in both the functions ``on_stop_button_clicked()`` and ``cancel_goal()``, called by the stop button of the first modality interface and all the five buttons of the second.  
The plot is not updated live, so the cell below needs to be run to show the current state.

In [18]:
# Bar plot
labels = ['reached','canceled']
eConf = {'ecolor': '0.3'}

fig3, ax3 = plt.subplots()
plt.bar(0, reached, align='edge', color='g')
plt.bar(1, canceled, align='edge', color='r')

plt.xlabel('Targets')
plt.ylabel('Number of times')
plt.title('Number of reached and canceled targets')
plt.xticks( np.arange(2)+0.45, labels)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Histogram of goal reaching time
The histogram below shows the time needed for the different goals to be reached. The graph plots the list of values ``elapsed_t`` which is updated with a new value every time a goal is reached in the function ``result_callback()``.  
As the one above, this plot is not updated live, so the cell below needs to be run to show the current state.

In [19]:
# Elapsed time histogram
global elapsed_t      # list of the amount of time taken to reach each goal 

fig2, ax2 = plt.subplots()
ax2.hist(elapsed_t)
ax2.set_title('Required time to reach a target')
ax2.set_ylabel('Elapsed time')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Final notes
The subscription to the topics ``/odom`` and ``/cmd_vel`` has been declared more than once in different cells in order to keep a certain level of modularity throughout the notebook. Contiguous cells need to be run toghether, as well as dependent blocks. The first cell containing the libraries, the global variables and the node start needs to be run in any case.